<a href="https://colab.research.google.com/github/yaobviously/sym-cargo/blob/main/sym_cargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install plotly-express --quiet
!pip install vincenty --quiet

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from vincenty import vincenty
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import plotly_express as px

In [6]:
port_file = '/content/drive/MyDrive/Ports/ports.csv'
tracking_file = '/content/drive/MyDrive/Ports/tracking.csv'

In [7]:
def wrangle():
  
  df1 = pd.read_csv(port_file)

  # converting lat and long to radians to compute haversine distance
  df1['lat_rad'] = np.radians(df1['lat'])
  df1['long_rad'] = np.radians(df1['long'])
  
  # rounding lat and long in port df
  df1['lat'] = df1['lat'].round(1)
  df1['long'] = df1['long'].round(1)

  df2 = pd.read_csv(tracking_file, parse_dates=['datetime'])
  df2 = df2.drop_duplicates()
  df2 = df2.sort_values(['vessel', 'datetime'])
  df2['vessel_1back'] = df2['vessel'].shift()
  
  # converting lat and long to radians to compute haversine distance  
  df2['lat_rad'] = np.radians(df2['lat'])
  df2['long_rad'] = np.radians(df2['long'])

  # adding lat/long column and lat/long 1 back to later compute delta
  df2['lat_long'] = [[x,y] for x, y in zip(df2['lat'], df2['long'])]
  df2['lat_long_1back'] = df2.groupby('vessel')['lat_long'].transform(lambda x: x.shift())   
  
  # time deltas to compute impossible distances travelled
  df2['time_delta'] = df2.groupby('vessel')['datetime'].transform(lambda x: x - x.shift(1))
  df2['hour_delta'] = df2['time_delta'].apply(lambda x: x.seconds / 3600)

  # filtering using query to eliminate unneeded/impossible values
  df2 = df2.query('hour_delta > 0.25 & speed <30 & heading <=360 & draft < 12.5')
  df2 = df2.reset_index(drop=True)

  return df1, df2

In [8]:
df1, df2 = wrangle()

In [9]:
def vincent_distance(row):
  """ 
  returns the vincenty distance for contiguous rows - will be used to identify
  impossible distances travelled, and so on. could be used to create distance
  matrix, but this may not be worthwhile 
  """
  if row['vessel'] != row['vessel_1back']:
    return -99

  loc1 = row['lat_long']
  loc2 = row['lat_long_1back']

  try:
    distance = vincenty(loc1, loc2)
    return distance
  except:
    return -99

In [10]:
# creating a dictionary of ports with their lat and longs - will be used to
# repeatedly to explore data and assign values

ports = {port:(lat, long) for port, lat, long in zip(df1['port'], df1['lat'], df1['long'])}

# a dictionary to retrieve the port id from the index
idx_ports = {idx:port for idx, port in zip(df1.index, df1.port)}

In [ ]:
# training a nearest neighbor model to  find the closest port when the 
# conditions indicating an extended stop have occurred. the metric is haversine
# in order to compute the 'great circle' distance. so i don't forget, the model
# returns the *index* of the port, not the port's identifying label

ports_train = df1[['lat_rad', 'long_rad']]

neigh_ports = NearestNeighbors(n_neighbors=3, algorithm='ball_tree', metric='haversine')
neigh_ports.fit(ports_train)

dist, n = neigh_ports.kneighbors(np.array([0.677565, 0.469731]).reshape(1,-1))

print(dist[0] * 6370)
print([idx_ports[n] for n in n[0]])

In [12]:
def nearest_port(df, radius=0.01):
  """
  returns the port identifier of the nearest port using the nearest neighbors
  model 
  """

  data = np.array([df['lat_rad'], df['long_rad']]).reshape(1, -1)
  dist, pred = neigh_ports.radius_neighbors(data, radius=radius, sort_results = True) 

  if len(dist[0]) == 0:
    return -1
  
  else:
    return idx_ports[pred[0][0]]

In [13]:
def nearest_distance(df, radius=0.0125):
  """
  returns the distance of the nearest port in the dataset
  """
  
  data = np.array([df['lat_rad'], df['long_rad']]).reshape(1, -1)
  dist, pred = neigh_ports.radius_neighbors(data, radius=radius, sort_results = True) 

  if len(dist[0]) == 0:
    return -1
  
  else:
    return dist[0][0]


In [14]:
def vincenty_port(row):
  """
  a function that computes the vincenty distance between the assigned port
  and the latitude and longitude of the location data. 
  """
  port = row['port_coords']
  loc = row['lat_long']
  return vincenty(port, loc)

In [40]:
def train_dbscan(df=df1, eps=0.1, min_samples=2):
  """
  use the dbscan clustering algorithm to find groupss of ports

  params
  ------
      df: pandas df
      eps: min distance between points in cluster
      min_samples: min members in a cluster

  returns
  -------
      labels: labels matching index of long/lat input pairs
  """
  
  coords = df[['long_rad', 'lat_rad']].values
  db = DBSCAN(eps=eps, min_samples=min_samples, algorithm='ball_tree', metric='haversine')

  db.fit(coords)

  return db.labels_

In [78]:
df1['labels_1'] = train_dbscan()

In [18]:
def assign_ports(df):
  """
  a function that prepares a dataframe for predictions

  Parameters:
  ----------
        df: pandas dataframe
  
  Returns:
  -------
        df: processed pandas dataframe

  """

  # resampling the data to standardize time intervals
  df = df.resample('4H').mean().ffill()
  df['lat_rad'] = df['lat_rad'].resample('4H').median().ffill()
  df['long_rad'] = df['long_rad'].resample('4H').median().ffill()
  
  # applying a mask to limit the rows the predict function is applied to
  mask = (df['vin_per_hour'] <= 0.75)
  df_temp = df[mask]

  df['pred_port'] = 0
  df.loc[mask, 'pred_port'] = df_temp.apply(nearest_port, axis=1)  
  df['port_coords'] = [list(ports[k]) if k in ports else -99 for k in df['pred_port']]
  df['lat_long'] = [[x,y] for x, y in zip(df['lat'], df['long'])]

  # applying a mask to limit the rows vincenty func is applied to
  vin_mask = (df['pred_port'] > 0)
  vin_temp = df[vin_mask]
  
  df['port_dist'] = 0
  df.loc[vin_mask, 'port_dist'] = vin_temp.apply(vincenty_port, axis=1)

  # eliminating entries in sequences where max distance greatly differs from min
  # indicates 'waiting to enter port'
  df['seq'] = df['pred_port'].diff().ne(0).cumsum()
  df['seq_count'] = df.groupby('seq')['seq'].transform('count')
  df['dist_diff'] = df.groupby('seq')['port_dist'].transform(lambda x: x - x.min())
  df['pred_port'] = np.where(df['dist_diff'] >= 15, 0, df['pred_port'])         # 15km away from closest point this trip to the port 

  df['draft_delta'] = df['draft'].transform(lambda x: abs(x.diff(-1)).ge(0.5))
  df['sum_draft_delta'] = df.groupby('seq')['draft_delta'].transform(sum)

  # recasting vessel and pred_port columns as integers
  df[['vessel', 'pred_port']] = df[['vessel', 'pred_port']].astype(int)

  return df

In [19]:
def get_voyages(df):
  """
  a function to convert the port sequences in each dataframe into voyages
  with the proper formatting

  Parameters:
  ----------
      df: pandas DataFrame
  
  Returns:
  -------
      df: processed pandas DataFrame
      
  """
  # filtering out columns without an assigned port
  nz = df[(df['pred_port'] > 0) | (df['pred_port'] == -75)].reset_index()

  vessel = nz['vessel'][0]
  dt = nz['datetime']
  pred = nz['pred_port']

  records = []

  for i in range(len(dt)-1):
    if pred[i] != pred[i+1]:
      start_port = pred[i]
      end_port = pred[i+1]
      begin_date = dt[i]
      end_date = dt[i+1]
      records.append([vessel, begin_date, end_date, start_port, end_port])

  df = pd.DataFrame.from_records(records, columns = ['vessel', 'begin_date', 'end_date', 'begin_port_id', 'end_port_id'])
  
  return df

In [14]:
df2['vin_diff'] = df2.apply(vincent_distance, axis=1)
df2['vin_per_hour'] = df2['vin_diff'] / df2['hour_delta']
df2 = df2.query('vin_per_hour <= 50')

In [20]:
# creating a dictionary of vessel dataframes with a datetime index

vessel_dfs = {}

for vessel in df2.vessel.unique():
  df_ = df2[df2['vessel'] == vessel]
  vessel_dfs[vessel] = df_.set_index('datetime')

# assigning ports to each dataframe
processed_dfs = {f'vessel_{key}':assign_ports(vessel_dfs[key]) for key in vessel_dfs.keys()}

In [21]:
voyages_df = pd.concat([get_voyages(processed_dfs[key]) for key in processed_dfs.keys()])
voyages_df['len_voyage'] = voyages_df['end_date'] - voyages_df['begin_date']

In [ ]:
voyages_df.sort_values(by='len_voyage', ascending=False).head(30)

In [23]:
alldf = pd.concat(processed_dfs[key] for key in processed_dfs.keys())

In [24]:
def prepare_data(df, n_input = 3):
  """
  preparing the sequences for models. it was setup to be flexible but i chipped
  away at it until it returned data for an xgboost 
  """

  df = get_voyages(df)
  vessel = df['vessel'].iloc[0]
  ports_ = np.array(df['begin_port_id'].append(pd.Series(df['end_port_id'].iloc[-1])))  

  X = []
  Y = []
  start = 0

  for i in range(len(ports_)):
    last_input = start + n_input
    last_output = last_input + 3
    if last_output <= len(ports_):
      x = ports_[start:last_input]
      y = ports_[last_input: last_output]
      X.append(x)
      Y.append(y)
      start += 1
  try:
    df = pd.concat([pd.DataFrame(X),
                  pd.DataFrame(Y, columns=['port_1ahead', 'port_2ahead', 'port_3ahead'])], axis=1)
    
  except:
    df = pd.DataFrame()
    
  # X = []

  # for x in X:
  #   for n in x:
  #     if n == -75:
  #       port_coords = [33, 140]
  #     else:
  #       port_coords = list(ports[n])
  #     port = [n]
  #     port.extend(port_coords)
  #     new_X.append(port)
  
  df['vessel'] = len(df) * [vessel]
  
  return df.astype(int)

In [ ]:
df = prepare_data(processed_dfs['vessel_133'], 6)

In [ ]:
dfs = [prepare_data(processed_dfs[key], 6) for key in processed_dfs.keys()]
model_df = pd.concat(dfs)

In [ ]:
alldf = pd.concat([processed_dfs[key] for key in processed_dfs.keys()])

In [ ]:
seq_2 = alldf[alldf['seq_count'] == 2]
seq_2

In [ ]:
unports = alldf[alldf['pred_port'] == -75]
unports['mdl'] = unports.apply(nearest_distance, axis=1)

unports.vin_per_hour.median()

In [ ]:
# creating a dictionary of dataframes for each port and then extracting the min,
# max, and max_draft distances for predictions from the dataframe

ports_dfs = {}

for port in alldf.pred_port.unique():
  ports_dfs[port] = alldf[alldf['pred_port'] == port]

del ports_dfs[-1]
del ports_dfs[0]

min_max = {}

for df in ports_dfs.keys():
  draft_max = ports_dfs[df].query('draft_delta >=1').port_dist.max()
  min_max[df] = [ports_dfs[df]['port_dist'].max(), ports_dfs[df]['port_dist'].min(), draft_max]

In [ ]:
all_vessels = pd.concat([processed_dfs[key] for key in processed_dfs.keys()])

In [ ]:
dist_check = all_vessels.groupby(['vessel', 'seq'], as_index=False)[['dist_diff', 'seq_count', 'pred_port']].apply(lambda x: x.max())

In [ ]:
fig = px.scatter_geo(unports, lat='lat', lon='long')

fig.show()

In [ ]:
# the dataframes to submit

voyages = pd.DataFrame(columns= ['vessel', 'begin_date', 'end_date', 'begin_port_id', 'end_port_id'])
predict = pd.DataFrame(columns= ['vessel', 'begin_port_id', 'end_port_id', 'voyage'])